<h1>Problem 1</h1>

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
To create the above dataframe:

1)The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

2)Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that - M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the - neighborhoods separated with a comma as shown in row 11 in the above table.

3)If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

4)Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

5)In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
   
   


In [1]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

     |████████████████████████████████| 102kB 7.5MB/s eta 0:00:01
     |████████████████████████████████| 92kB 7.5MB/s eta 0:00:01
     |████████████████████████████████| 829kB 26.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Packages installed.


In [2]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 4.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [5]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [15]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood


In [16]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)

neighbors.head(10)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,M2A\n
1,M3A\n,North York\n,Parkwoods\n
2,M4A\n,North York\n,Victoria Village\n
3,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
4,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
5,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
6,M8A\n,Not assigned\n,M9A\n
7,M1B\n,Scarborough\n,"Malvern, Rouge\n"
8,M2B\n,Not assigned\n,M3B\n
9,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens\n"


In [17]:
neighbors.shape

(135, 3)